In [1]:
#hiddencell
# from pbl_tools import *

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

fe = fm.FontEntry(fname = 'MaruBuri-Regular.otf', name = 'MaruBuri')
fm.fontManager.ttflist.insert(0, fe)
plt.rc('font', family='MaruBuri')

# **대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [12]:
import os
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(77) 

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [3]:
import pandas as pd 

train_org = pd.read_csv('./data/train.csv') 
test_org = pd.read_csv('./data/test.csv')

train_org.drop(['ID'], axis=1, inplace=True)
test_org.drop(['ID'], axis=1, inplace=True)

display(train_org.head())
print(train_org.info())
print(train_org.isnull().sum())
print('<=============================================================================>')
display(test_org.tail())
print(test_org.info())
print(test_org.isnull().sum())

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,...,상해없음,보행자,남,38세,경상,0,0,1,0,3
3,2019-01-01 02,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,...,상해없음,승용,남,36세,중상,0,1,0,0,5
4,2019-01-01 04,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,...,상해없음,승용,남,52세,경상,0,0,1,0,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   사고일시         39609 non-null  object
 1   요일           39609 non-null  object
 2   기상상태         39609 non-null  object
 3   시군구          39609 non-null  object
 4   도로형태         39609 non-null  object
 5   노면상태         39609 non-null  object
 6   사고유형         39609 non-null  object
 7   사고유형 - 세부분류  39609 non-null  object
 8   법규위반         39609 non-null  object
 9   가해운전자 차종     39609 non-null  object
 10  가해운전자 성별     39609 non-null  object
 11  가해운전자 연령     39609 non-null  object
 12  가해운전자 상해정도   39609 non-null  object
 13  피해운전자 차종     38618 non-null  object
 14  피해운전자 성별     38618 non-null  object
 15  피해운전자 연령     38618 non-null  object
 16  피해운전자 상해정도   38618 non-null  object
 17  사망자수         39609 non-null  int64 
 18  중상자수         39609 non-null  int64 
 19  경상자수         39609 non-nu

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
10958,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차
10962,2022-12-31 21,토요일,맑음,대구광역시 동구 효목동,교차로 - 교차로부근,건조,차대차


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   사고일시    10963 non-null  object
 1   요일      10963 non-null  object
 2   기상상태    10963 non-null  object
 3   시군구     10963 non-null  object
 4   도로형태    10963 non-null  object
 5   노면상태    10963 non-null  object
 6   사고유형    10963 non-null  object
dtypes: object(7)
memory usage: 599.7+ KB
None
사고일시    0
요일      0
기상상태    0
시군구     0
도로형태    0
노면상태    0
사고유형    0
dtype: int64


## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [4]:
display(f"train : {train_org.iloc[0]['사고일시']} ~ {train_org.iloc[-1]['사고일시']}")
display(f"test : {test_org.iloc[0]['사고일시']} ~ {test_org.iloc[-1]['사고일시']}")     

'train : 2019-01-01 00 ~ 2021-12-31 23'

'test : 2022-01-01 01 ~ 2022-12-31 21'

## **예측 대상 확인하기**

sample_submission.csv를 로드하여 출력 형식을 확인해 봄으로서, 
대회 페이지 규칙에 기술되어 있는, 예측 대상이 무엇인지 확인해 봅시다.

최종적으로 예측해야 하는 값은 'ECLO'(인명피해 심각도) 값입니다. 

그러나, train데이터 컬럼에 있는 '사망자수', '중상자수', '경상자수', '부상자수'를 각각 예측하여 ECLO 예측값을 구할 수도 있습니다. 즉, ECLO 값을 예측하는 단일 모델로 접근할 수도,  각각을 별도로 예측하는 모델로 접근할 수도 있습니다. 

<img src="https://drive.google.com/uc?id=19PvSqu8k04D3a-Bskcl-d-Xkckkn2Fvv">  

In [5]:
sample_submission = pd.read_csv("./data/sample_submission.csv")
sample_submission.head()

,ID,ECLO
0,ACCIDENT_39609,0
1,ACCIDENT_39610,0
2,ACCIDENT_39611,0
3,ACCIDENT_39612,0
4,ACCIDENT_39613,0


## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [6]:
train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})' 

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다 
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다 

# 해당 과정을 test_x에 대해서도 반복해줍니다 
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(train_df)
display(f"columns of train_df : {train_df.columns}")
display(test_df)
display(f"columns of test_df : {test_df.columns}")

,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,...,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,연,월,일,시간
0,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,...,중상,0,1,0,0,5,2019,1,1,0
1,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,...,경상,0,0,1,0,3,2019,1,1,0
2,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,...,경상,0,0,1,0,3,2019,1,1,1
3,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,...,중상,0,1,0,0,5,2019,1,1,2
4,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,...,경상,0,0,1,0,3,2019,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,승용,여,...,경상,0,0,1,0,3,2021,12,31,19
39605,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,여,...,경상,0,0,1,0,3,2021,12,31,19
39606,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,승용,남,...,중상,0,2,0,0,10,2021,12,31,21
39607,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,...,경상,0,0,1,0,3,2021,12,31,22


"columns of train_df : Index(['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

,요일,기상상태,시군구,도로형태,노면상태,사고유형,연,월,일,시간
0,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,2022,1,1,1
1,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,2022,1,1,1
2,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,2022,1,1,4
3,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차,2022,1,1,4
4,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차,2022,1,1,6
...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차,2022,12,31,18
10959,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차,2022,12,31,18
10960,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차,2022,12,31,20
10961,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차,2022,12,31,20


"columns of test_df : Index(['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

## **파생 변수 생성 2 : 공간(위치) 정보 생성** 

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [7]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(train_df)
display(f"columns of train_df : {train_df.columns}")
display(test_df)
display(f"columns of test_df : {test_df.columns}")

,요일,기상상태,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,경상자수,부상자수,ECLO,연,월,일,시간,도시,구,동
0,화요일,맑음,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,...,0,0,5,2019,1,1,0,대구광역시,중구,대신동
1,화요일,흐림,단일로 - 기타,건조,차대사람,보도통행중,기타,승용,남,39세,...,1,0,3,2019,1,1,0,대구광역시,달서구,감삼동
2,화요일,맑음,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,...,1,0,3,2019,1,1,1,대구광역시,수성구,두산동
3,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,49세,...,0,0,5,2019,1,1,2,대구광역시,북구,복현동
4,화요일,맑음,단일로 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,30세,...,1,0,3,2019,1,1,4,대구광역시,동구,신암동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,승용,여,52세,...,1,0,3,2021,12,31,19,대구광역시,수성구,수성동3가
39605,금요일,맑음,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,...,1,0,3,2021,12,31,19,대구광역시,달서구,상인동
39606,금요일,맑음,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,...,0,0,10,2021,12,31,21,대구광역시,달서구,월성동
39607,금요일,맑음,기타 - 기타,건조,차대차,추돌,안전운전불이행,승용,남,40세,...,1,0,3,2021,12,31,22,대구광역시,달서구,장동


"columns of train_df : Index(['요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

,요일,기상상태,도로형태,노면상태,사고유형,연,월,일,시간,도시,구,동
0,토요일,맑음,교차로 - 교차로안,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동
1,토요일,맑음,단일로 - 기타,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동
2,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,4,대구광역시,수성구,수성동2가
3,토요일,맑음,단일로 - 기타,건조,차대차,2022,1,1,4,대구광역시,수성구,신매동
4,토요일,맑음,교차로 - 교차로안,건조,차대차,2022,1,1,6,대구광역시,달서구,감삼동
...,...,...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,단일로 - 터널,건조,차대차,2022,12,31,18,대구광역시,남구,대명동
10959,토요일,맑음,단일로 - 기타,건조,차대차,2022,12,31,18,대구광역시,수성구,시지동
10960,토요일,맑음,단일로 - 기타,건조,차대차,2022,12,31,20,대구광역시,수성구,연호동
10961,토요일,맑음,교차로 - 교차로부근,건조,차대차,2022,12,31,20,대구광역시,수성구,범물동


"columns of test_df : Index(['요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구',\n       '동'],\n      dtype='object')"

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [8]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(train_df)
display(f"columns of train_df : {train_df.columns}")
display(test_df)
display(f"columns of test_df : {test_df.columns}")

,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,ECLO,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,...,5,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,화요일,흐림,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,...,3,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,...,3,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,...,5,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,...,3,2019,1,1,4,대구광역시,동구,신암동,단일로,기타
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,금요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,52세,상해없음,...,3,2021,12,31,19,대구광역시,수성구,수성동3가,교차로,교차로안
39605,금요일,맑음,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,상해없음,...,3,2021,12,31,19,대구광역시,달서구,상인동,단일로,기타
39606,금요일,맑음,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,중상,...,10,2021,12,31,21,대구광역시,달서구,월성동,교차로,교차로안
39607,금요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,40세,상해없음,...,3,2021,12,31,22,대구광역시,달서구,장동,기타,기타


"columns of train_df : Index(['요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동,교차로,교차로안
1,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동,단일로,기타
2,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,수성동2가,교차로,교차로안
3,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,신매동,단일로,기타
4,토요일,맑음,건조,차대차,2022,1,1,6,대구광역시,달서구,감삼동,교차로,교차로안
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,건조,차대차,2022,12,31,18,대구광역시,남구,대명동,단일로,터널
10959,토요일,맑음,건조,차대차,2022,12,31,18,대구광역시,수성구,시지동,단일로,기타
10960,토요일,맑음,건조,차대차,2022,12,31,20,대구광역시,수성구,연호동,단일로,기타
10961,토요일,맑음,건조,차대차,2022,12,31,20,대구광역시,수성구,범물동,교차로,교차로부근


"columns of test_df : Index(['요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다 

In [14]:
display(train_df.head())
display(test_df.head())

,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,...,ECLO,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,...,5,2019,1,1,0,대구광역시,중구,대신동,단일로,기타
1,화요일,흐림,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,...,3,2019,1,1,0,대구광역시,달서구,감삼동,단일로,기타
2,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,...,3,2019,1,1,1,대구광역시,수성구,두산동,단일로,기타
3,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,...,5,2019,1,1,2,대구광역시,북구,복현동,단일로,기타
4,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,...,3,2019,1,1,4,대구광역시,동구,신암동,단일로,기타


,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,상동,교차로,교차로안
1,토요일,맑음,건조,차대사람,2022,1,1,1,대구광역시,수성구,지산동,단일로,기타
2,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,수성동2가,교차로,교차로안
3,토요일,맑음,건조,차대차,2022,1,1,4,대구광역시,수성구,신매동,단일로,기타
4,토요일,맑음,건조,차대차,2022,1,1,6,대구광역시,달서구,감삼동,교차로,교차로안


## **train, test 데이터의 독립변수, 종속 변수 분리**

모델 추론 시점에서 획득할 수 있는 데이터는 학습 시점에서 활용할 수 있는 데이터와 다를 수 있습니다  
그렇기에 본 baseline에서는 test 시점에서만 활용할 수 있는 데이터들(test의 column들)만 사용하여 train 데이터를 구성했습니다 

train_df 중에서 test_x가 가진 column들만 추출하여 train_x(독립변수)를 구성하고, train_df 중에서 예측 목표인 EPDO를 추출하여 train_y(종속변수)를 구성합니다

In [10]:
X_test = test_df.copy()
X_train = train_df[X_test.columns].copy()
y_train = train_df['ECLO'].copy()

## **범주형(Categorical) 변수, 수치형 변수로 변환하기**

모델 학습을 위해 train_x의 문자열 타입의 컬럼들을 추출하고, LabelEncoder를 활용하여 이 컬럼들을 모두 수치형 변수로 변환해 보겠습니다

In [11]:
for i in X_train.columns:
    print(i, X_train[i].nunique())
    print(i, X_train[i].unique())
    print(X_train.info())
    print(i, X_test[i].nunique())
    print(i, X_test[i].unique())
    print(X_test.info())
    print()

요일 7
요일 ['화요일' '수요일' '목요일' '금요일' '토요일' '일요일' '월요일']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요일      39609 non-null  object
 1   기상상태    39609 non-null  object
 2   노면상태    39609 non-null  object
 3   사고유형    39609 non-null  object
 4   연       39609 non-null  int64 
 5   월       39609 non-null  int64 
 6   일       39609 non-null  int64 
 7   시간      39609 non-null  int64 
 8   도시      39609 non-null  object
 9   구       39609 non-null  object
 10  동       39609 non-null  object
 11  도로형태1   39609 non-null  object
 12  도로형태2   39609 non-null  object
dtypes: int64(4), object(9)
memory usage: 3.9+ MB
None
요일 7
요일 ['토요일' '일요일' '월요일' '화요일' '수요일' '목요일' '금요일']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요일    

In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# list(X_train.dtypes[X_train.dtypes == "object"].index) : ['요일', '기상상태', '노면상태', '사고유형', '도시', '구', '동', '도로형태1', '도로형태2']
# 도시는 대구광역시로 동일하기 때문에 제거
# 연도별 추세는 중요하지 않다고 판단하여 제거
X_train.drop(['도시', '연'], axis=1, inplace=True)
X_test.drop(['도시', '연'], axis=1, inplace=True)
categorical_features = ['요일', '기상상태', '노면상태', '사고유형', '구', '동', '도로형태1', '도로형태2']
# 추출된 문자열 변수 확인
data = pd.concat([X_train, X_test])

# from sklearn.preprocessing import LabelEncoder
for i in categorical_features:
    # le = LabelEncoder()
    # le=le.fit(X_train[i]) 
    # X_train[i]=le.transform(X_train[i])
    
    # for case in np.unique(X_test[i]):
    #     if case not in le.classes_: 
    #         le.classes_ = np.append(le.classes_, case) 
    # X_test[i]=le.transform(X_test[i])

    # Get dummies
    dummies = pd.get_dummies(data[i], prefix=i)
    # Drop the original column
    data.drop([i], axis=1, inplace=True)
    # Concatenate the dummy variables
    data = pd.concat([data, dummies], axis=1)

# Split the data back into train and test dataframes
X_train = data[:len(X_train)]
X_test = data[len(X_train):]

display(X_train.head())
display(X_test.head())

,월,일,시간,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,도로형태2_고가도로위,도로형태2_교량,도로형태2_교차로부근,도로형태2_교차로안,도로형태2_교차로횡단보도내,도로형태2_기타,도로형태2_미분류,도로형태2_주차장,도로형태2_지하차도(도로)내,도로형태2_터널
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,1,1,2,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,1,1,4,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


,월,일,시간,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,도로형태2_고가도로위,도로형태2_교량,도로형태2_교차로부근,도로형태2_교차로안,도로형태2_교차로횡단보도내,도로형태2_기타,도로형태2_미분류,도로형태2_주차장,도로형태2_지하차도(도로)내,도로형태2_터널
0,1,1,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,1,4,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,4,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1,1,6,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


## Evaluation

GradientBoostingRegressor, RandomForest, XGBoost, LGBM

In [16]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [30]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_log_error
from math import sqrt

# Define RMSLE function
def rmsle(y_true, y_pred):
    return sqrt(mean_squared_log_error(y_true, y_pred))

# Fit & Predict Model
# Gradient Boosting
gb_model = GradientBoostingRegressor()
gb_model.fit(X_tra, y_tra)
gb_preds = gb_model.predict(X_val)
# Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_tra, y_tra)
rf_preds = rf_model.predict(X_val)
# Xgboost
xgb_model = XGBRegressor()
xgb_model.fit(X_tra, y_tra)
xgb_preds = xgb_model.predict(X_val)
# LightGBM
lgb_model = LGBMRegressor(max_depth=5, n_estimators=100)
lgb_model.fit(X_tra, y_tra)
lgb_preds = lgb_model.predict(X_val)


# Calculate RMSLE
gb_score = rmsle(y_val, gb_preds)
print(f'GradientBoosting RMSLE: {round(gb_score, 4)}')
rf_score = rmsle(y_val, rf_preds)
print(f'RandomForest RMSLE: {round(rf_score, 4)}')
xgb_score = rmsle(y_val, xgb_preds) 
print(f'XGBoost RMSLE: {round(xgb_score, 4)}')
lgb_score = rmsle(y_val, lgb_preds)
print(f'LightGBM RMSLE: {round(lgb_score, 4)}')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000848 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 429
[LightGBM] [Info] Number of data points in the train set: 31687, number of used features: 183
[LightGBM] [Info] Start training from score 4.730710
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

Seed = 77

Only get_dummies 
- GradientBoosting : 0.459
- RandomForest : 0.4814
- XGBoost : 0.4645
- LGBM : 0.4592

After Standard Scaling 
- RandomForest : 
- XGBoost : 
- LGBM : 



GridSearchCV

In [33]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'learning_rate': [0.01, 0.1, 1]
}

# Initialize the GridSearchCV
grid_search = GridSearchCV(GradientBoostingRegressor(), 
                           param_grid, 
                           cv=10, 
                           scoring='neg_mean_squared_log_error',
                           )

# Fit the GridSearchCV
grid_search.fit(X_tra, y_tra)

# Get the best parameters and the best score
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

After GridSearchCV
- LGBM : 0.4592 (max_depth=5, n_estimators=100)

## **Model Train & Prediction** 

전체 학습 데이터에 대하여 학습후 최종 예측을 합니다. 여기서는 DecisionTreeRegressor 모델로 학습하고 예측하는 예시 코드입니다.

In [117]:
# Final Model
model = LGBMRegressor(max_depth=5, n_estimators=100, random_state=7) 
model.fit(X_train, y_train)

prediction = model.predict(X_test)
prediction

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000947 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 435
[LightGBM] [Info] Number of data points in the train set: 39609, number of used features: 186
[LightGBM] [Info] Start training from score 4.726704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

array([4.89619759, 4.02794758, 6.04610207, ..., 4.96844638, 5.06336034,
       5.13113592])

## **Submission 양식 확인**

sample_submission.csv 화일 데이터(sample_submission)를 그대로 복사한 후, 
양식의 'ECLO' 컬럼에 test_x에 대한 ECLO(y) 예측값을 입력합니다 

In [118]:
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = prediction
baseline_submission 

,ID,ECLO
0,ACCIDENT_39609,4.896198
1,ACCIDENT_39610,4.027948
2,ACCIDENT_39611,6.046102
3,ACCIDENT_39612,5.519666
4,ACCIDENT_39613,5.078919
...,...,...
10958,ACCIDENT_50567,6.867764
10959,ACCIDENT_50568,4.786608
10960,ACCIDENT_50569,4.968446
10961,ACCIDENT_50570,5.063360


## **답안지 저장 및 제출하기**

In [119]:
baseline_submission.to_csv('lgbm2_submit.csv', index=False)